## Plotly Maps

In [9]:
import pandas as pd

%config Completer.use_jedi = False
pd.options.mode.chained_assignment = None  # default='warn'

In [10]:
import geopandas as gpd
import matplotlib.pyplot as plt
import geopy as gp
import numpy as np

data_location = 'properati_final_maps.csv'

data = pd.read_csv(data_location, encoding ='latin1')


In [11]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

In [12]:
# Convertimos el DataFrame en un GeoDataFrame agregando la columna geometry con las coordenadas de las propiedades.
geo_dataset = gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data.lon, data.lat))

In [13]:
#Genero el dataset de caba
geo_dataset_caba = geo_dataset[geo_dataset["state_name"] == "Capital Federal"]

In [14]:
# Reemplazo algunos barrios por los barrios oficiales de CABA definidos por el Gobierno de la Ciudad

geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['LAS CAÑITAS'],'PALERMO')
geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['BARRIO NORTE'],'RECOLETA')
geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['CONGRESO'],'BALVANERA')
geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['CONSTITUCIÓN'],'CONSTITUCION')
geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['POMPEYA'],'NUEVA POMPEYA')
geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['AGRONOMÍA'],'AGRONOMIA')
geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['VILLA PUEYRREDÓN'],'VILLA PUEYRREDON')
geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['PARQUE CENTENARIO'],'CABALLITO')
geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['ABASTO'],'BALVANERA')
geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['SAN NICOLÁS'],'SAN NICOLAS')
geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['CENTRO / MICROCENTRO'],'SAN NICOLAS')
geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['ONCE'],'BALVANERA')
geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['TRIBUNALES'],'SAN NICOLAS')
geo_dataset_caba['place_name'] = geo_dataset_caba['place_name'].replace(['VILLA GENERAL MITRE'],"VILLA GRAL. MITRE")

In [15]:
#Importo el archivo JSON de CABA

from urllib.request import urlopen
import json
with urlopen('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/barrios/barrios.geojson') as response:
    counties = json.load(response)

In [16]:
barrio_map = {}

for feature in counties['features']:
    feature['id'] = feature['properties']['BARRIO']
    barrio_map[feature['properties']['BARRIO']] = feature['id']

In [17]:
geo_dataset_caba['id'] = geo_dataset_caba['place_name'].apply(lambda x: barrio_map[x])

In [18]:
# Genero la figura para preciom2

fig = px.choropleth(geo_dataset_caba, locations = 'id', geojson = counties, color = 'price_m2_calc_supTotal', scope='south america' )
fig.update_geos(fitbounds = 'locations', visible = False)
fig.show()

In [19]:
fig.write_html("plotly.html")

In [20]:
# Genero la figura para precio total

fig = px.choropleth(geo_dataset_caba, locations = 'id', geojson = counties, color = 'price_aprox_usd', scope='south america' )
fig.update_geos(fitbounds = 'locations', visible = False)
fig.show()


	